In [6]:
import nltk
import re 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.tokenize import RegexpTokenizer
import math
import distance as d
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.tokenize import RegexpTokenizer
import string
import nltk
import spacy
import re
import json
from flask import jsonify

In [7]:
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english')) 
spacy.prefer_gpu()
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jono/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [8]:
text1="""
"title" : "Remstal Gartenschau is looking for a professional with event experience",
    "companyName" : "Building Authority Stuttgart",
    "timeFrame" : {
       "startDate": "2018-12-06T16:04:31.423Z",
       "endDate": "2019-01-31T16:04:31.423Z"
    },
    "employment" : {
       "type" : "part-time",
       "frequency": "2 times per week"
    },
    "location" :"Stuttgart",
    "language": ["German"],
    "teamSetup" : "Team of 10 people working part-time",
    "experience": "Events, Communications, Marketing",
    "travelling": "Only within Baden-Wurttemberg",
    "overview": "The Remstal Gartenschau is a horticultural show that takes place from 10.05. until 20.10.2019 in the state of Baden-Wurttemberg and includes 16 cities and municipalities. The goal is to promote green space in cities and its surroundings. Many different initiatives will take place throughout this time in order to enable visitors a wonderful experience.",
    "responsibilities": "We are looking for an experienced professional (we gladly accept the support of retired professionals/senior experts) that autonomously can support all event-related preparations and also implement an organizational support system for all events happening during the event time. Your tasks include communication with all relevant stakeholders, determine the event schedule and organize all event details. A team of 9 will support you throughout the project duration."

"""

In [31]:
#This function removes special characters from a string
def remove_string_special_characters(s):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens= tokenizer.tokenize(s)
    return preprocess(" ".join(tokens))

In [32]:
def preprocess(sentence):
    sentence = sentence
    #tokenizer = RegexpTokenizer(r'\w+')
    word_tokens = word_tokenize(sentence)
    filtered_words = [w for w in word_tokens if not w in stopwords.words('english')]
    
    return " ".join(filtered_words)

In [33]:
def get_doc(sent):
    """this function splits the text into sentences and considering each sentence as a document, 
    calculates the total word count of each"""
    doc_info=[]
    i=0
    for sent in text_sents_clean:
        i+=1
        count=count_words(sent)
        temp={'doc_id':1,'doc_length':count}
        doc_info.append(temp)
    return doc_info

In [34]:
def count_words(sent):
    "this function return the total number of words in the input text"
    count=0
    words=word_tokenize(sent)
    for word in words:
        count+=1
    return count

In [35]:
def create_freq_dict(sents):
    """
    This function creates a frequency dictionary for each word in each document
    """
    i=0
    freqDict_list=[]
    for sent in sents:
        i+=1
        freq_dict={}
        words=word_tokenize(sent)
        for word in words:
            word=word.lower()
            if word in freq_dict:
                freq_dict[word]+=1
            else:
                freq_dict[word]=1
            temp={'doc_id':i, 'freq_dict':freq_dict}
        freqDict_list.append(temp)
    return freqDict_list

In [36]:
def computeTF(doc_info,freqDict_list):
    """tf=(frequency of the term in the doc/total number of terms in the doc)"""
    TF_scores=[]
    for tempDict in freqDict_list:
        id=tempDict['doc_id']
        for k in tempDict['freq_dict']:
            temp={'doc_id':id, 'TF_score':tempDict['freq_dict'][k]/doc_info[id-1]['doc_length'], 'key':k}
            TF_scores.append(temp)
    return TF_scores

In [37]:
def computeIDF(doc_info,freqDict_list):
    """
    idf=ln(total number of docs/number of docs with term in it)
    """
    IDF_scores=[]
    counter=0
    for dict in freqDict_list:
        counter+=1
        for k in dict['freq_dict'].keys():
            count=sum([k in tempDict['freq_dict'] for tempDict in freqDict_list])
            temp={'doc_id':counter, 'IDF_score':math.log(len(doc_info)/count), 'key':k}
            
            IDF_scores.append(temp)
    return IDF_scores

In [38]:
def computeTFIDF(TF_scores,IDF_scores):
    """tf=(frequency of the term in the doc/total number of terms in the doc)"""
    TFIDF_scores=[]
    for j in IDF_scores:
        for i in TF_scores:
            if j['key']==i['key'] and j['doc_id']==i['doc_id']:
                temp={'doc_id':j['IDF_score']*i['TF_score'],'key':i['key']}
            TFIDF_scores.append(temp)
    return TFIDF_scores

In [39]:
text_sents=sent_tokenize(text1)
text_sents_clean=[remove_string_special_characters(s) for s in text_sents]
text_sents_clean

['title Remstal Gartenschau looking professional event experience companyName Building Authority Stuttgart timeFrame startDate 2018 12 06T16 04 31 423Z endDate 2019 01 31T16 04 31 423Z employment type part time frequency 2 times per week location Stuttgart language German teamSetup Team 10 people working part time experience Events Communications Marketing travelling Only within Baden Wurttemberg overview The Remstal Gartenschau horticultural show takes place 10 05 20 10 2019 state Baden Wurttemberg includes 16 cities municipalities',
 'The goal promote green space cities surroundings',
 'Many different initiatives take place throughout time order enable visitors wonderful experience',
 'responsibilities We looking experienced professional gladly accept support retired professionals senior experts autonomously support event related preparations also implement organizational support system events happening event time',
 'Your tasks include communication relevant stakeholders determine e

In [40]:
doc_info=get_doc(text_sents_clean)
doc_info

[{'doc_id': 1, 'doc_length': 75},
 {'doc_id': 1, 'doc_length': 7},
 {'doc_id': 1, 'doc_length': 12},
 {'doc_id': 1, 'doc_length': 26},
 {'doc_id': 1, 'doc_length': 12},
 {'doc_id': 1, 'doc_length': 7}]

In [41]:
freqDict_list=create_freq_dict(text_sents_clean)
TF_scores=computeTF(doc_info,freqDict_list)
IDF_scores=computeIDF(doc_info,freqDict_list)

In [42]:
IDF_scores

[{'doc_id': 1, 'IDF_score': 1.791759469228055, 'key': 'title'},
 {'doc_id': 1, 'IDF_score': 1.791759469228055, 'key': 'remstal'},
 {'doc_id': 1, 'IDF_score': 1.791759469228055, 'key': 'gartenschau'},
 {'doc_id': 1, 'IDF_score': 1.0986122886681098, 'key': 'looking'},
 {'doc_id': 1, 'IDF_score': 1.0986122886681098, 'key': 'professional'},
 {'doc_id': 1, 'IDF_score': 0.6931471805599453, 'key': 'event'},
 {'doc_id': 1, 'IDF_score': 1.0986122886681098, 'key': 'experience'},
 {'doc_id': 1, 'IDF_score': 1.791759469228055, 'key': 'companyname'},
 {'doc_id': 1, 'IDF_score': 1.791759469228055, 'key': 'building'},
 {'doc_id': 1, 'IDF_score': 1.791759469228055, 'key': 'authority'},
 {'doc_id': 1, 'IDF_score': 1.791759469228055, 'key': 'stuttgart'},
 {'doc_id': 1, 'IDF_score': 1.791759469228055, 'key': 'timeframe'},
 {'doc_id': 1, 'IDF_score': 1.791759469228055, 'key': 'startdate'},
 {'doc_id': 1, 'IDF_score': 1.791759469228055, 'key': '2018'},
 {'doc_id': 1, 'IDF_score': 1.791759469228055, 'key': 